# Print Metrics

## Creating Required Functions

### Import Library

In [176]:
from pathlib import Path
import os
import pandas as pd

### Function to Find the Best YOLO Model (best.pt)

In [177]:
def find_best_model(base_dir='runs_yolo/'):
    best_paths = list(Path(base_dir).rglob('best.pt'))
    if not best_paths:
        raise FileNotFoundError("No 'best.pt' file found in the 'runs/' directory.")
    
    # Optionally, sort by latest modified time
    best_paths.sort(key=lambda p: p.stat().st_mtime, reverse=True)
    
    print(f"[+] Found best.pt at: {best_paths[0]}")
    return str(best_paths[0])


### Functions to Load and Print Metrics from YOLO Training Results

In [178]:
def find_results_csv(directory):
    """Find the results.csv file in the specified directory."""
    for root, dirs, files in os.walk(directory):
        if 'results.csv' in files:
            return os.path.join(root, 'results.csv')
    return None

def load_results_csv(results_csv_path):
    """Load the results CSV into a pandas DataFrame."""
    return pd.read_csv(results_csv_path)

def calculate_total_epochs(df):
    """Calculate the total number of epochs from the DataFrame."""
    return df['epoch'].max()

def calculate_training_loss(epoch_data):
    """Calculate the total training loss from the given epoch data."""
    train_box_loss = epoch_data['train/box_loss']
    train_cls_loss = epoch_data['train/cls_loss']
    train_dfl_loss = epoch_data['train/dfl_loss']
    return train_box_loss + train_cls_loss + train_dfl_loss

def calculate_validation_loss(epoch_data):
    """Calculate the total validation loss from the given epoch data."""
    val_box_loss = epoch_data['val/box_loss']
    val_cls_loss = epoch_data['val/cls_loss']
    val_dfl_loss = epoch_data['val/dfl_loss']
    return val_box_loss + val_cls_loss + val_dfl_loss

def print_final_metrics(df):
    """Print the final metrics for the last epoch."""
    final_epoch_data = df.iloc[-1]

    # Calculate total training and validation loss
    train_loss = calculate_training_loss(final_epoch_data)
    val_loss = calculate_validation_loss(final_epoch_data)

    # Print overall metrics
    print("\n========== Final Training Metrics ==========")
    print(f"Training Loss: {train_loss:.6f}")
    print(f"Precision: {final_epoch_data['metrics/precision(B)']:.6f}")
    print(f"Recall: {final_epoch_data['metrics/recall(B)']:.6f}")
    print(f"mAP@0.5: {final_epoch_data['metrics/mAP50(B)']:.6f}")
    print(f"mAP@0.5:0.95: {final_epoch_data['metrics/mAP50-95(B)']:.6f}")

    print("\n========== Final Validation Metrics ==========")
    print(f"Validation Loss: {val_loss:.6f}")


def print_csv_metrics(directory):
    """Main function to process and print final metrics."""
    # Find the results.csv file
    results_csv_path = find_results_csv(directory)
    
    if not results_csv_path:
        print("Error: 'results.csv' file not found in the specified directory.")
        return

    print(f"Found results.csv at: {results_csv_path}")

    # Load results CSV
    df = load_results_csv(results_csv_path)

    # Get the total number of epochs
    total_epochs = calculate_total_epochs(df)
    print(f"Total number of epochs: {total_epochs}")

    print_final_metrics(df)


### Function to Compare Metrics Between Two YOLO Training Runs

In [179]:
import pandas as pd

def compare_final_metrics(csv1_path, csv2_path):
    df1 = pd.read_csv(csv1_path)
    df2 = pd.read_csv(csv2_path)

    last1 = df1.iloc[-1]
    last2 = df2.iloc[-1]

    metrics_to_compare = {
        "train/box_loss": "Box Loss (Train)",
        "train/cls_loss": "Cls Loss (Train)",
        "train/dfl_loss": "DFL Loss (Train)",
        "metrics/precision(B)": "Precision",
        "metrics/recall(B)": "Recall",
        "metrics/mAP50(B)": "mAP@0.5",
        "metrics/mAP50-95(B)": "mAP@0.5:0.95",
        "val/box_loss": "Box Loss (Val)",
        "val/cls_loss": "Cls Loss (Val)",
        "val/dfl_loss": "DFL Loss (Val)"
    }

    print("Changes in Metrics Before and After Retraning:\n")
    print(f"{'Metric':<25} {'Before':<10} {'After':<10} {'Diff':<10} {'Trend'}")
    print("-" * 65)

    for key, label in metrics_to_compare.items():
        val1 = last1[key]
        val2 = last2[key]
        diff = val2 - val1
        if abs(diff) > 1e-6:
            # If increase, color green; if decrease, color red
            if diff > 0:
                trend = f"\033[92m Increase\033[0m"  # Green
            else:
                trend = f"\033[91m Decrease\033[0m"  # Red

            # Printing with colors
            print(f"{label:<25} {val1:<10.5f} {val2:<10.5f} {diff:<10.5f} {trend}")


### Function to Compare mAP@0.5:0.95 Metrics Between Two Models (Before and After Retraining)

In [1]:
import json

def compare_maps(json_path1, json_path2):
    with open(json_path1, 'r') as f1, open(json_path2, 'r') as f2:
        metrics1 = json.load(f1)
        metrics2 = json.load(f2)

    print("\nmAP@0.5:0.95 Differences Before and After Retraning:\n")
    print(f"{'Class':<15} {'Before':<10} {'After':<10} {'Diff':<10} {'Trend'}")
    print("-" * 60)

    for class_name in metrics1:
        map1 = metrics1[class_name].get("mAP@0.5:0.95", 0)
        map2 = metrics2.get(class_name, {}).get("mAP@0.5:0.95", 0)

        diff = map2 - map1
        if abs(diff) > 1e-6:
            if diff > 0:
                trend = f"\033[92m Increase\033[0m"  # Green for increase
            else:
                trend = f"\033[91m Decrease\033[0m"  # Red for decrease

            # Printing with colors
            print(f"{class_name:<15} {map1:<10.4f} {map2:<10.4f} {diff:<10.4f} {trend}")


### YOLO Model Evaluation and Metrics Extraction

In [181]:
import json
from ultralytics import YOLO

def load_yolo_model(model_path):
    return YOLO(model_path)

def run_model_validation(model, yaml):
    return model.val(data=yaml)

def extract_per_class_metrics(results):
    """
    Extracts mAP@0.5:0.95 per class from results.
    NOTE: Only mAP@0.5:0.95 is available via `results.box.maps`
    """
    per_class_metrics = {}
    if hasattr(results.box, 'maps') and results.box.maps is not None:
        maps = results.box.maps  # This is a NumPy array [num_classes]
        for i, name in results.names.items():
            per_class_metrics[name] = {
                "class_id": i,
                "mAP@0.5:0.95": round(float(maps[i]), 4)
            }
    else:
        print("[-] No per-class mAP@0.5:0.95 data found.")
    return per_class_metrics

def save_metrics_to_json(metrics, output_path):
    with open(output_path, "w") as f:
        json.dump(metrics, f, indent=4)
    print(f"[+] Saved per-class metrics to {output_path}")

def evaluate_and_save_metrics(model_path, yaml, output_json_path="per_class_metrics.json"):
    model = load_yolo_model(model_path)
    results = run_model_validation(model, yaml)
    metrics = extract_per_class_metrics(results)
    save_metrics_to_json(metrics, output_json_path)

### Per-Class mAP@0.5:0.95 Metrics Extraction and Display

In [182]:
import json

def print_per_class_metrics(json_path="per_class_metrics.json"):
    with open(json_path, "r") as f:
        metrics = json.load(f)
    
    print("Per-Class mAP@0.5:0.95 Metrics:\n")
    print(f"{'Class Name':<15} {'Class ID':<10} {'mAP@0.5:0.95':<15}")
    print("-" * 40)
    
    for name, data in metrics.items():
        print(f"{name:<15} {data['class_id']:<10} {data['mAP@0.5:0.95']:<15}")


## Calling Functions

### Before Retraning

In [183]:
yolov8 = './runs/train/yolov8'
print_csv_metrics(yolov8)

Found results.csv at: ./runs/train/yolov8/results.csv
Total number of epochs: 50

========== Final Training Metrics ==========
Training Loss: 2.047820
Precision: 0.643800
Recall: 0.268230
mAP@0.5: 0.282050
mAP@0.5:0.95: 0.177310

========== Final Validation Metrics ==========
Validation Loss: 5.496680


In [184]:
best_pt_path = find_best_model(yolov8)
evaluate_and_save_metrics(best_pt_path, yaml="yolov8.yaml",output_json_path="per_class_metrics.json")

[+] Found best.pt at: runs/train/yolov8/weights/best.pt
Ultralytics 8.3.109 🚀 Python-3.10.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3050 OEM, 7957MiB)


Model summary (fused): 72 layers, 3,008,378 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/ssl49/Desktop/Automated-Labeling-for-Aerial-Images-main/Automated_SegmentAndYolo/datasets/new_dataset_yolo_split/val/labels.cache... 1181 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1181/1181 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 74/74 [00:10<00:00,  7.16it/s]


                   all       1181      70393      0.577       0.27       0.29      0.186
                  pool          9          9      0.757      0.889      0.889      0.731
            vegetation         75       1780      0.155      0.254      0.149     0.0798
                  roof         42         79     0.0743      0.696      0.527      0.377
                  wall         62        255      0.067      0.208     0.0843     0.0513
                window         38        139      0.527      0.201      0.243      0.152
                person         75        637       0.82      0.262      0.369      0.188
                   dog          6         12          1          0    0.00567    0.00295
                   car       1115      53669      0.796      0.289      0.491      0.246
               bicycle         32         53      0.477       0.31      0.326      0.103
                  tree         38        108      0.869      0.389      0.512      0.356
                 truc

In [185]:
print_per_class_metrics("per_class_metrics.json")

Per-Class mAP@0.5:0.95 Metrics:

Class Name      Class ID   mAP@0.5:0.95   
----------------------------------------
unlabeled       0          0.1855         
pool            1          0.7305         
vegetation      2          0.0798         
roof            3          0.3768         
wall            4          0.0513         
window          5          0.1524         
person          6          0.1884         
dog             7          0.0029         
car             8          0.2459         
bicycle         9          0.1027         
tree            10         0.3563         
truck           11         0.0291         
bus             12         0.0            
vehicle         13         0.0955         


### After Retraning

In [186]:
new_path = './runs/train/fine-tune-yolov8'
print_csv_metrics(new_path)

Found results.csv at: ./runs/train/fine-tune-yolov8/results.csv
Total number of epochs: 11

========== Final Training Metrics ==========
Training Loss: 2.637500
Precision: 0.627330
Recall: 0.537950
mAP@0.5: 0.553620
mAP@0.5:0.95: 0.399380

========== Final Validation Metrics ==========
Validation Loss: 2.982540


In [187]:
best_pt_path = find_best_model(new_path)
evaluate_and_save_metrics(best_pt_path, yaml="yolo_retrain.yaml",output_json_path="per_class_metrics_retrain.json")

[+] Found best.pt at: runs/train/fine-tune-yolov8/weights/best.pt
Ultralytics 8.3.109 🚀 Python-3.10.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3050 OEM, 7957MiB)


Model summary (fused): 72 layers, 3,008,378 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/ssl49/Desktop/Automated-Labeling-for-Aerial-Images-main/Automated_SegmentAndYolo/datasets/split_videos_dataset/val/labels.cache... 109 images, 0 backgrounds, 0 corrupt: 100%|██████████| 109/109 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:01<00:00,  5.18it/s]


                   all        109        421      0.101      0.378      0.108     0.0715
            vegetation         44         55     0.0415      0.473      0.041     0.0208
                  roof         12         13     0.0171      0.154     0.0121    0.00811
                person         15         15     0.0723        0.2     0.0466     0.0166
                   car         74        281      0.192      0.648      0.179      0.111
                  tree         24         33          0          0     0.0805     0.0636
               vehicle         16         24      0.281      0.792       0.29      0.209
Speed: 0.9ms preprocess, 3.3ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/val2
[+] Saved per-class metrics to per_class_metrics_retrain.json


In [188]:
print_per_class_metrics("per_class_metrics_retrain.json")

Per-Class mAP@0.5:0.95 Metrics:

Class Name      Class ID   mAP@0.5:0.95   
----------------------------------------
unlabeled       0          0.0715         
pool            1          0.0715         
vegetation      2          0.0208         
roof            3          0.0081         
wall            4          0.0715         
window          5          0.0715         
person          6          0.0166         
dog             7          0.0715         
car             8          0.1107         
bicycle         9          0.0715         
tree            10         0.0636         
truck           11         0.0715         
bus             12         0.0715         
vehicle         13         0.2094         


### Compare both metrics

In [189]:
new_path = './runs/train/fine-tune-yolov8'
old_path = './runs/train/yolov8'

results_csv_path = find_results_csv(new_path)
results_csv_path_1 = find_results_csv(old_path)

compare_final_metrics(results_csv_path_1, results_csv_path)

Changes in Metrics Before and After Retraning:

Metric                    Before     After      Diff       Trend
-----------------------------------------------------------------
Box Loss (Train)          0.73327    0.98916    0.25589     Increase
Cls Loss (Train)          0.45441    0.70747    0.25306     Increase
DFL Loss (Train)          0.86014    0.94087    0.08073     Increase
Precision                 0.64380    0.62733    -0.01647    Decrease
Recall                    0.26823    0.53795    0.26972     Increase
mAP@0.5                   0.28205    0.55362    0.27157     Increase
mAP@0.5:0.95              0.17731    0.39938    0.22207     Increase
Box Loss (Val)            2.02042    0.99684    -1.02358    Decrease
Cls Loss (Val)            2.42410    0.97288    -1.45122    Decrease
DFL Loss (Val)            1.05216    1.01282    -0.03934    Decrease


In [4]:
compare_maps("per_class_metrics.json", "per_class_metrics_retrain.json")


mAP@0.5:0.95 Differences Before and After Retraning:

Class           Before     After      Diff       Trend
------------------------------------------------------------
unlabeled       0.1855     0.0715     -0.1140     Decrease
pool            0.7305     0.0715     -0.6590     Decrease
vegetation      0.0798     0.0208     -0.0590     Decrease
roof            0.3768     0.0081     -0.3687     Decrease
wall            0.0513     0.0715     0.0202      Increase
window          0.1524     0.0715     -0.0809     Decrease
person          0.1884     0.2166     0.0282      Increase
dog             0.0029     0.0715     0.0686      Increase
car             0.2459     0.3107     0.0648      Increase
bicycle         0.1027     0.0715     -0.0312     Decrease
tree            0.3563     0.0636     -0.2927     Decrease
truck           0.0291     0.0715     0.0424      Increase
bus             0.0000     0.0715     0.0715      Increase
vehicle         0.0955     0.2094     0.1139      Increase
